In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-28 08:46:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.57MB/s    in 0.2s    

2022-07-28 08:46:42 (5.57 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles
import pandas as pd
url = "https://raw.githubusercontent.com/jsguti323/Housing_Estimator/main/Data/Clean_Census.csv"
clean_census_df = pd.read_csv(url)
clean_census_df.head()

,County,Employees,Establishments
0,Alpine,1048,41
1,Amador,5985,782
2,Alameda,505595,32808
3,Butte,45746,4613
4,Calaveras,4781,870


In [10]:
url = "https://raw.githubusercontent.com/jsguti323/Housing_Estimator/main/Data/Clean_Housing.csv"
clean_housing_df = pd.read_csv(url)
clean_housing_df.head()

,City,Longitude,Lattitude,Population,Median Age,Median Income,Median House Value,Total Rooms,Bedrooms,Households,Ocean Proximity
0,Berkeley,-122.23,37.88,322.0,41.0,8.3252,452600.0,880.0,129.0,126.0,NEAR BAY
1,Piedmont,-122.22,37.86,2401.0,21.0,8.3014,358500.0,7099.0,1106.0,1138.0,NEAR BAY
2,Piedmont,-122.24,37.85,496.0,52.0,7.2574,352100.0,1467.0,190.0,177.0,NEAR BAY
3,Berkeley,-122.25,37.85,558.0,52.0,5.6431,341300.0,1274.0,235.0,219.0,NEAR BAY
4,Berkeley,-122.25,37.85,565.0,52.0,3.8462,342200.0,1627.0,280.0,259.0,NEAR BAY


In [11]:
url = "https://raw.githubusercontent.com/jsguti323/Housing_Estimator/main/Data/Clean_Population.csv"
clean_population_df = pd.read_csv(url)
clean_population_df.head()

,County,City,Population
0,San Bernardino,Adelanto,8517
1,Los Angeles,Agoura Hills,20390
2,Alameda,Alameda,76459
3,Alameda,Albany,16327
4,Los Angeles,Alhambra,82106


In [12]:
url = "https://raw.githubusercontent.com/jsguti323/Housing_Estimator/main/Data/Clean_Weather.csv"
clean_weather_df = pd.read_csv(url)
clean_weather_df.head()

,City,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Berkeley,88.09,65,20,10.36,few clouds
1,Piedmont,87.76,62,20,10.36,few clouds
2,Oakland,87.69,63,75,17.27,broken clouds
3,Orinda,86.95,59,0,12.66,clear sky
4,Alameda,91.31,39,0,8.01,clear sky


In [16]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://housingdata.c6ie1cdxfwsy.us-west-1.rds.amazonaws.com/HousingData"
config = {"user":"postgres", 
          "password": "Housingdata", 
          "driver":"org.postgresql.Driver"}

In [17]:
# Write review_id_df to table in RDS
spark_clean_census_df = spark.createDataFrame(clean_census_df)
spark_clean_census_df.write.jdbc(url=jdbc_url, table='Census', mode=mode, properties=config)

In [18]:
# Write review_id_df to table in RDS
spark_clean_housing_df = spark.createDataFrame(clean_housing_df)
spark_clean_housing_df.write.jdbc(url=jdbc_url, table='Housing', mode=mode, properties=config)

In [19]:
# Write review_id_df to table in RDS
spark_clean_population_df = spark.createDataFrame(clean_population_df)
spark_clean_population_df.write.jdbc(url=jdbc_url, table='Population', mode=mode, properties=config)

In [20]:
# Write review_id_df to table in RDS
spark_clean_weather_df = spark.createDataFrame(clean_weather_df)
spark_clean_weather_df.write.jdbc(url=jdbc_url, table='Weather', mode=mode, properties=config)